<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/Sort_Abfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


try: shutil.rmtree('/content/CrispyMice')
except: None
!git clone https://github.com/dtabuena/CrispyMice
to_import = ['CrispyPipelineFuncs.ipynb']
for i in to_import:
    f = '/content/CrispyMice/' + i
    %run $f


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'EphysLib'...
remote: Enumerating objects: 589, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 589 (delta 199), reused 170 (delta 125), pack-reused 307
Receiving objects: 100% (589/589), 14.25 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (373/373), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'CrispyMice'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 260 (delta 106), reused 25 (delta 25), pack-reused 107
Receiving objects: 100% (260/260), 41.89 MiB | 9.10 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [9]:
def reorg_abfs(new_dir='reorg', local_abf_dir='/content/my_ephys_data'): #look_up_file, 
    # rootdir = '/content/my_ephys_data_Fast_Data'
    # new_dir = 'reorg'
    # lut_df = pd.read_csv('/content/Fast_Data_Recording_LookUp.csv')
    try: shutil.rmtree('/content/'+new_dir)
    except: None
    os.mkdir('/content/'+new_dir)
    for subdir, dirs, fils in os.walk(local_abf_dir):
        for file in fils:
            if '.abf' in file:
                full_path = (os.path.join(subdir, file))
                abf_file = abf_or_name(full_path)
                prot_name = abf_file.protocol
                new_prot_dir = new_dir+'/'+prot_name
                try: os.mkdir('/content/'+new_prot_dir)
                except: None
                shutil.copyfile(full_path, new_prot_dir+'/'+file)
    shutil.make_archive(new_dir, 'zip', new_dir)
    from google.colab import files
    files.download(new_dir+'.zip')
    return None


In [ ]:
dataset = {'data_name': 'RNF182',
           'data_source': "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }
file_loc = get_drobox_folder(dataset['data_source'], 'my_ephys_data_' + dataset['data_name'])
reorg_abfs(new_dir=dataset['data_name']+'_reorg', local_abf_dir=file_loc)


dataset = {'data_name': 'MGDA2',
           'data_source': "https://www.dropbox.com/sh/bj5ke9c1pqwpsct/AACqznMXzJj7epVBVDaSifnma?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }
file_loc = get_drobox_folder(dataset['data_source'], 'my_ephys_data_' + dataset['data_name'])
reorg_abfs(new_dir=dataset['data_name']+'_reorg', local_abf_dir=file_loc)


dataset = {'data_name': 'NELL2',
           'data_source': "https://www.dropbox.com/sh/a3vrgx8n2lvs53w/AAAVi1gSXYaP3gY4PmvWlRbAa?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }
file_loc = get_drobox_folder(dataset['data_source'], 'my_ephys_data_' + dataset['data_name'])
reorg_abfs(new_dir=dataset['data_name']+'_reorg', local_abf_dir=file_loc)

In [11]:
# def reorg_abfs(new_dir='reorg', local_abf_dir='/content/my_ephys_data'): #look_up_file, 
#     # rootdir = '/content/my_ephys_data_Fast_Data'
#     # new_dir = 'reorg'
#     # lut_df = pd.read_csv('/content/Fast_Data_Recording_LookUp.csv')
#     try: shutil.rmtree('/content/'+new_dir)
#     except: None
#     os.mkdir('/content/'+new_dir)
#     for subdir, dirs, fils in os.walk(local_abf_dir):
#         for file in fils:
#             if '.abf' in file:
#                 full_path = (os.path.join(subdir, file))
#                 abf_file = abf_or_name(full_path)
#                 prot_name = abf_file.protocol
#                 new_prot_dir = new_dir+'/'+prot_name
#                 try: os.mkdir('/content/'+new_prot_dir)
#                 except: None
#                 shutil.copyfile(full_path, new_prot_dir+'/'+file)
#     shutil.make_archive(new_dir, 'zip', new_dir)
#     from google.colab import files
#     files.download(new_dir+'.zip')
#     return None


# data_source = dataset['data_source']
# data_name = dataset['data_name']
# file_naming_scheme = dataset['file_naming_scheme']
# results = {}
# reorg_abfs(new_dir=data_name+'_reorg', local_abf_dir=file_loc)

